In [10]:
import pandas


file_path = "../dataset"
files = ("connections", "devices", "processes", "profiles")

dataset = {}
for file in files:
    dataset[file] = pandas.read_csv(f"{file_path}/{file}.csv", sep="\t")

Connections description


In [11]:
dataset["connections"].info()
dataset["connections"].describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15108 entries, 0 to 15107
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ts                 15108 non-null  object 
 1   imei               15108 non-null  int64  
 2   mwra               15108 non-null  float64
 3   c.dogalize         15108 non-null  float64
 4   c.android.gm       15108 non-null  float64
 5   c.android.youtube  15108 non-null  float64
 6   c.android.chrome   15108 non-null  float64
 7   c.katana           15108 non-null  float64
 8   c.raider           15108 non-null  float64
 9   c.android.vending  15108 non-null  float64
 10  c.UCMobile.x86     15108 non-null  float64
 11  c.updateassist     15108 non-null  float64
 12  c.UCMobile.intl    15108 non-null  float64
dtypes: float64(11), int64(1), object(1)
memory usage: 1.5+ MB


,imei,mwra,c.dogalize,c.android.gm,c.android.youtube,c.android.chrome,c.katana,c.raider,c.android.vending,c.UCMobile.x86,c.updateassist,c.UCMobile.intl
count,1.510800e+04,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000
mean,3.934258e+18,0.625563,10.309496,10.928002,12.141882,10.933567,10.416465,48.846209,49.482347,49.685602,49.822994,49.805628
std,3.332767e+18,0.483993,2.626325,2.696930,2.225872,2.652465,2.626934,13.031848,28.905578,28.962924,29.030406,28.843199
min,3.590434e+17,0.000000,0.642300,2.587630,2.348110,2.655890,1.165660,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.630331e+17,0.000000,8.434330,8.943805,10.934585,8.965822,8.538965,39.977420,24.570853,24.731063,24.532995,24.735187
50%,3.590434e+18,1.000000,10.154350,10.844220,12.375635,10.780055,10.263805,48.962015,49.269510,49.300740,49.852925,49.520235
75%,8.630331e+18,1.000000,12.049473,12.850918,13.642702,12.806403,12.192747,57.737270,74.651803,74.873525,75.301215,74.849797
max,8.630331e+18,1.000000,20.212930,22.004980,19.587410,21.092210,19.919510,100.000000,100.000000,100.000000,100.000000,100.000000


In [12]:
dataset["connections"].head()

,ts,imei,mwra,c.dogalize,c.android.gm,c.android.youtube,c.android.chrome,c.katana,c.raider,c.android.vending,c.UCMobile.x86,c.updateassist,c.UCMobile.intl
0,2018-05-05 10:00:00,3590433799317661693,1.0,5.11208,10.02817,15.07671,12.46608,8.10118,46.20406,20.71493,79.76850,75.68132,63.99591
1,2018-05-05 10:01:00,863033069630348362,1.0,12.53455,12.44584,13.20187,9.40848,7.27400,47.72439,25.30530,80.52045,62.36753,93.24322
2,2018-05-05 10:02:00,863033069630348040,1.0,10.72532,10.75620,10.77881,11.11719,12.79590,25.14114,77.40480,37.28781,46.97291,9.08001
3,2018-05-05 10:03:00,8630330696303482121,0.0,7.73201,9.52005,12.24554,11.12170,13.77184,48.52576,72.40145,9.99980,64.95956,84.85341
4,2018-05-05 10:04:00,3590433799317661495,0.0,6.61303,7.32034,14.42927,8.01345,13.95424,64.73747,70.80009,94.57614,71.02708,27.98753


Devices description


In [13]:
dataset["devices"].info()
dataset["devices"].describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2929 entries, 0 to 2928
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   latitude    2929 non-null   float64
 1   longitude   2929 non-null   float64
 2   store_name  2929 non-null   object 
 3   code        2929 non-null   object 
 4   location    2929 non-null   object 
 5   imei        2929 non-null   int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 137.4+ KB


,latitude,longitude,imei
count,2929.000000,2929.000000,2.929000e+03
mean,29.279310,11.548748,3.872383e+18
std,22.929049,73.811177,3.312779e+18
min,-54.800000,-156.474320,3.590434e+17
25%,15.460630,-59.133160,8.630331e+17
50%,36.192780,9.476290,3.590434e+18
75%,46.083330,73.000000,8.630331e+18
max,68.798330,178.004170,8.630331e+18


In [14]:
dataset["devices"].head()

,latitude,longitude,store_name,code,location,imei
0,48.21644,9.02596,Albstadt,DE,Europe/Berlin,863033069630348990
1,12.74409,-8.07257,Kati,ML,Africa/Bamako,3590433799317661677
2,7.83389,-72.47417,Villa del Rosario,CO,America/Bogota,3590433799317661990
3,12.37169,123.62494,Masbate,PH,Asia/Manila,3590433799317661172
4,39.33427,-76.43941,Middle River,US,America/New_York,863033069630348073


Processes description


In [15]:
dataset["processes"].info()
dataset["processes"].describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15108 entries, 0 to 15107
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ts                          15108 non-null  object 
 1   imei                        15108 non-null  int64  
 2   mwra                        15108 non-null  float64
 3   p.android.packageinstaller  15108 non-null  float64
 4   p.android.settings          15108 non-null  float64
 5   p.android.documentsui       15108 non-null  float64
 6   p.android.chrome            15108 non-null  float64
 7   p.android.gm                15108 non-null  float64
 8   p.system                    15108 non-null  float64
 9   p.android.externalstorage   15108 non-null  float64
 10  p.android.vending           15108 non-null  float64
 11  p.process.gapps             15108 non-null  float64
 12  p.google                    15108 non-null  float64
 13  p.browser.provider          151

,imei,mwra,p.android.packageinstaller,p.android.settings,p.android.documentsui,p.android.chrome,p.android.gm,p.system,p.android.externalstorage,p.android.vending,...,p.browser.provider,p.android.defcontainer,p.dogalize,p.olauncher,p.simulator,p.inputmethod.latin,p.android.gms,p.notifier,p.katana,p.gms.persistent
count,1.510800e+04,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,...,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000,15108.000000
mean,3.934258e+18,0.625563,11.572954,11.499864,10.310165,9.790817,12.771425,10.397043,11.576398,0.062140,...,50.072203,47.326426,50.037140,49.640124,50.022306,50.260806,50.272805,49.751673,49.852339,49.573461
std,3.332767e+18,0.483993,2.636035,2.679572,2.648527,2.436871,1.816167,2.664877,2.713959,1.030622,...,13.310724,13.104973,28.833214,28.910236,28.957691,28.843929,28.820311,28.685114,28.967083,29.068352
min,3.590434e+17,0.000000,2.487240,1.570430,2.279980,0.000000,5.552490,0.616400,2.292350,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.630331e+17,0.000000,9.739860,9.656940,8.417333,8.122188,11.547503,8.453595,9.704448,0.000240,...,41.272058,38.443713,25.007917,24.370213,24.879258,25.193368,25.569652,25.352768,24.522633,24.168370
50%,3.590434e+18,1.000000,11.703395,11.619575,10.138660,9.641190,12.781690,10.351115,11.693555,0.001460,...,49.965620,47.353845,50.400705,49.551330,50.338015,50.040285,50.465125,49.859050,49.514105,49.986190
75%,8.630331e+18,1.000000,13.443035,13.414413,12.119515,11.364935,14.001280,12.327390,13.534700,0.010613,...,58.825585,56.251165,75.002775,74.764725,74.949060,75.316412,75.275442,74.062057,75.035590,74.764915
max,8.630331e+18,1.000000,20.415500,20.927500,20.122050,19.855360,19.625710,19.216320,21.071240,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [16]:
dataset["processes"].head()

,ts,imei,mwra,p.android.packageinstaller,p.android.settings,p.android.documentsui,p.android.chrome,p.android.gm,p.system,p.android.externalstorage,...,p.browser.provider,p.android.defcontainer,p.dogalize,p.olauncher,p.simulator,p.inputmethod.latin,p.android.gms,p.notifier,p.katana,p.gms.persistent
0,2018-05-05 10:00:00,3590433799317661693,1.0,9.23642,14.64737,12.28429,8.10855,11.56273,9.81747,7.96281,...,35.91940,24.95922,21.17604,80.95317,64.67547,55.80378,82.99368,96.45875,95.47350,18.60413
1,2018-05-05 10:01:00,863033069630348362,1.0,12.12539,13.31378,8.53914,8.99158,10.84797,10.73896,12.39776,...,59.79664,48.67044,40.08113,25.00209,86.68513,4.17599,65.28470,11.04057,72.81380,55.76611
2,2018-05-05 10:02:00,863033069630348040,1.0,11.93971,13.62370,13.71940,10.16446,14.61026,10.14790,10.00803,...,37.66953,51.76377,87.50645,8.40823,32.66810,96.13825,4.88761,29.88836,82.48309,18.89299
3,2018-05-05 10:03:00,8630330696303482121,0.0,11.75678,5.40945,10.37504,12.96885,15.71257,13.59185,14.10555,...,47.54273,51.31861,40.91248,46.38488,35.03554,24.37016,69.40371,27.45343,53.48129,80.62817
4,2018-05-05 10:04:00,3590433799317661495,0.0,13.29218,9.88140,10.48243,12.30196,10.07931,14.95220,12.61291,...,44.97460,35.66277,43.71764,14.70324,73.82801,24.01651,55.68464,29.72480,93.90266,79.73910


Profiles description


In [17]:
dataset["profiles"].info()
dataset["profiles"].describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2571 entries, 0 to 2570
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   username      2571 non-null   object
 1   address       2185 non-null   object
 2   name          2571 non-null   object
 3   user_id       2571 non-null   int64 
 4   job           771 non-null    object
 5   ssn           2571 non-null   object
 6   company       2571 non-null   object
 7   imei          2571 non-null   int64 
 8   registration  2571 non-null   object
 9   residence     900 non-null    object
 10  mail          2571 non-null   object
 11  birthdate     1414 non-null   object
dtypes: int64(2), object(10)
memory usage: 241.2+ KB


,user_id,imei
count,2571.000000,2.571000e+03
mean,1294.560482,3.848391e+18
std,734.696063,3.307759e+18
min,0.000000,3.590434e+17
25%,660.500000,8.630331e+17
50%,1280.000000,3.590434e+18
75%,1938.000000,8.630331e+18
max,2570.000000,8.630331e+18


In [18]:
dataset["profiles"].head()

,username,address,name,user_id,job,ssn,company,imei,registration,residence,mail,birthdate
0,nancywagner,Unit 3496 Box 9846\r\nDPO AE 23667,Shelly Thomas,1713,NaN,568-30-2539,"Miller, Kirby and Davis",863033069630348636,02 Aug 2018,NaN,brandonsmith@hotmail.com,2004-12-08
1,millerdakota,"677 Jay Vista Suite 015\r\nBrownhaven, LA 18432",Amy Horn,1886,NaN,395-74-2254,"Gonzalez, Andrade and Robinson",359043379931766007,18 Feb 2022,NaN,keithgibbs@gmail.com,1931-09-11
2,meyerabigail,"3746 Baldwin Estate Suite 335\r\nWest Travis, ...",Leah Campbell,184,NaN,848-91-2310,"Boyle, Taylor and Sampson",8630330696303481487,2021/08/17,NaN,steve72@yahoo.com,1931-11-04
3,timothyhart,"8748 Villarreal Lights\r\nWest Michaelfurt, WA...",Joseph Miller,920,Fast food restaurant manager,247-95-0332,Smith-Rodriguez,8630330696303481156,2023-10-24,"864 Jared Overpass Suite 869\r\nPort Amanda, W...",mfletcher@hotmail.com,1955-01-15
4,jenniferwilliams,"499 Michael Flat\r\nJesseview, HI 05817",Kimberly Williams,2557,NaN,162-02-9998,Ramsey-Fernandez,8630330696303481453,2018-03-19,NaN,ryanmartinez@yahoo.com,1966-09-30
